In [11]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
import arrayfire as af

from dg_maxwell import utils
from dg_maxwell import lagrange
from dg_maxwell import params

af.set_backend(params.backend)

In [12]:
# Prototype implementation for Lp_Li
N   = 3

N_a = 3
foo = af.range(N * N_a, dtype = af.Dtype.u32)
foo = af.moddims(foo, d0 = N, d2 = 1, d1 = N_a)

N_b = 3
bar = af.range(N * N_b, dtype = af.Dtype.u32)
bar = af.moddims(bar, d0 = N, d2 = 1, d1 = N_b)

foo_init = foo.copy()
bar_init = bar.copy()

# print(foo)
# print(bar)

foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N))
foo = af.moddims(foo, d0 = N * N, d1 = 1, d2 = N)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N * N))
foo = af.transpose(af.moddims(af.transpose(foo), d0 = N * N * N * N, d1 = 1, d2 = N))
foo = af.reorder(foo, d0 = 2, d1 = 1, d2 = 0)
print(foo.shape)

bar = af.reorder(bar, d0 = 0, d1 = 2, d2 = 1)
bar = af.transpose(af.tile(bar, d0 = 1, d1 = N))
bar = af.moddims(bar, d0 = N * N, d1 = 1, d2 = N)
bar = af.tile(bar, d0 = 1, d1 = N * N)
bar = af.moddims(af.transpose(bar), d0 = N * N * N * N, d1 = 1, d2 = N)
bar = af.reorder(af.transpose(bar), d0 = 2, d1 = 1, d2 = 0)
# print(af.transpose(bar))

foo_bar = af.transpose(af.convolve1(foo, bar, conv_mode = af.CONV_MODE.EXPAND))
# print(foo_bar)

(3, 81)


In [13]:
# Prototype implementation for Lq_Lj
N   = 3

N_a = 3
foo = af.range(N * N_a, dtype = af.Dtype.u32)
foo = af.moddims(foo, d0 = N, d2 = 1, d1 = N_a)

N_b = 3
bar = af.range(N * N_b, dtype = af.Dtype.u32)
bar = af.moddims(bar, d0 = N, d2 = 1, d1 = N_b)

# print(foo)
# print(bar)

foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
foo = af.tile(foo, d0 = 1, d1 = N)
foo = af.moddims(foo, d0 = N * N, d1 = 1, d2 = N)
foo = af.transpose(af.tile(foo, d0 = 1, d1 = N * N))
foo = af.moddims(af.transpose(foo), d0 = N * N * N * N, d1 = 1, d2 = N)
foo = af.reorder(foo, d0 = 2, d1 = 0, d2 = 1)
# print(foo.shape)

bar = af.reorder(bar, d0 = 0, d1 = 2, d2 = 1)
bar = af.tile(bar, d0 = 1, d1 = N)
bar = af.moddims(bar, d0 = N * N, d1 = 1, d2 = N)
bar = af.tile(bar, d0 = 1, d1 = N * N)
bar = af.moddims(af.transpose(bar), d0 = N * N * N * N, d1 = 1, d2 = N)
bar = af.reorder(af.transpose(bar), d0 = 2, d1 = 1, d2 = 0)
# print(bar)

foo_bar = af.transpose(af.convolve1(foo, bar, conv_mode = af.CONV_MODE.EXPAND))
print(foo_bar.shape)

(81, 5)


In [14]:
foo = af.reorder(foo, d0 = 0, d1 = 2, d2 = 1)
# foo = af.tile(foo, d0 = 1, d1 = N)
# foo = af.moddims()

In [15]:
print('Poly1d square using convolve1')
# print(af.transpose(af.convolve1(af.transpose(foo), af.transpose(bar), conv_mode = af.CONV_MODE.EXPAND)))
# print(utils.poly1d_product(foo, bar))
print(af.transpose(af.convolve1(af.transpose(foo),
                                af.transpose(bar),
                                conv_mode = af.CONV_MODE.EXPAND)))


Poly1d square using convolve1
arrayfire.Array()
Type: unsigned int

[3 81 81 1]
         0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0          0 
         9          9       